In [2]:
import joblib
import numpy as np 
import mne
import numpy as np
from scipy.stats import skew, kurtosis
from mne.preprocessing import ICA
from mne.time_frequency import psd_array_welch
import pandas as pd
from scipy.fftpack import fft
import pywt  # For wavelet transform

In [14]:
import numpy as np
import pandas as pd
import pywt
from scipy.stats import skew, kurtosis
from scipy.fftpack import fft

def extract_channel_features(raw, condition, fmin=0.5, fmax=50):
    # Select only EEG channels
    raw.pick('eeg')
    data = raw.get_data()
    channel_names = raw.ch_names
    features = {}  # Store condition as 0 for EO, 1 for EC
    
    # condition_prefix = 'eo' if condition == 0 else 'ec'
    
    # Time-domain features
    for i, ch in enumerate(channel_names):
        key_prefix = f'{condition}_{ch.lower()}'
        features[f'{key_prefix}_mean'] = np.mean(data[i])
        features[f'{key_prefix}_variance'] = np.var(data[i])
        features[f'{key_prefix}_skewness'] = skew(data[i])
        features[f'{key_prefix}_kurtosis'] = kurtosis(data[i])
        features[f'{key_prefix}_peak_to_peak'] = np.ptp(data[i])

        # Fourier Transform (FFT)
        fft_values = np.abs(fft(data[i]))
        features[f'{key_prefix}_fft_mean'] = np.mean(fft_values)
        features[f'{key_prefix}_fft_std'] = np.std(fft_values)
        features[f'{key_prefix}_fft_max'] = np.max(fft_values)

        # Wavelet Transform (DWT) using Daubechies wavelet (db4)
        coeffs = pywt.wavedec(data[i], 'db4', level=4)
        features[f'{key_prefix}_wavelet_energy'] = sum(np.sum(np.square(c)) for c in coeffs)
        
        wavelet_entropy = 0
        for c in coeffs:
            c = c[np.isfinite(c)]
            c_norm = c / (np.sum(np.abs(c)) + 1e-10)

            c_norm = c[c > 0]  # Remove zero or negative values to avoid log errors
            if len(c_norm) > 0:
                wavelet_entropy += -np.sum(c_norm * np.log2(c_norm))
            else:
                wavelet_entropy += 0  # If empty, set entropy to 0
        features[f'{key_prefix}_wavelet_entropy'] = wavelet_entropy
    
    # Frequency-domain features using PSD
    psd = raw.compute_psd(method='welch', fmin=fmin, fmax=fmax, n_fft=2048)
    psd_data = psd.get_data()
    freqs = psd.freqs
    psd_df = pd.DataFrame(psd_data, columns=freqs, index=channel_names)

    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'slow_alpha': (6, 9), 'alpha': (8, 12),
             'beta': (12, 30), 'gamma': (30, 50)}

    for band, (low, high) in bands.items():
        band_power = psd_df.loc[:, (freqs >= low) & (freqs <= high)].mean(axis=1)
        for ch in channel_names:
            key_prefix = f'{condition}_{ch.lower()}'
            features[f'{key_prefix}_{band}_power'] = band_power[ch]

    # Frontal Alpha Asymmetry (F3-F4)
    if 'F3' in channel_names and 'F4' in channel_names:
        features[f'{condition}f3_f4_alpha_asymmetry'] = features[f'{condition}_f4_alpha_power'] - features[f'{condition}_f3_alpha_power']

    # Convert features to DataFrame
    features_df = pd.DataFrame([features])

    return features_df


In [15]:
def process_and_combine(eo_file_path, ec_file_path, output_file):
    all_features = []

    # Process EO file
    raw_eo = mne.io.read_raw_fif(eo_file_path)
    features_eo = extract_channel_features(raw_eo,"ec")
    #features_eo['condition'] = 'EO'
    all_features.append(features_eo)

    # Process EC file
    raw_ec = mne.io.read_raw_fif(ec_file_path)
    features_ec = extract_channel_features(raw_ec,"eo")
    #features_ec['condition'] = 'EC'
    all_features.append(features_ec)

    # Combine EO and EC features
    import os
    combined_features = pd.concat(all_features,axis=1)
    # out_path = (out_dir,output_file)
    # Save combined features to a single CSV file
    combined_features.to_csv(output_file,index=False)
    print(f"Features successfully saved to {output_file}")
    # return combined_features

# single file usage

In [16]:
process_and_combine("/home/admincit/Desktop/Team_4/split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif","/home/admincit/Desktop/Team_4/split_fif/mdd/sub-88000489_ses-1_task-restEO_eeg_1.fif","preprocessed.csv")

Opening raw data file /home/admincit/Desktop/Team_4/split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 29)  idle
    Range : 0 ... 29999 =      0.000 ...    59.998 secs
Ready.
Effective window size : 4.096 (s)
Opening raw data file /home/admincit/Desktop/Team_4/split_fif/mdd/sub-88000489_ses-1_task-restEO_eeg_1.fif...
Isotrak not found
    Read a total of 1 projection items:
        Average EEG reference (1 x 29)  idle
    Range : 0 ... 29999 =      0.000 ...    59.998 secs
Ready.


/tmp/ipykernel_2210216/113133525.py:5: RuntimeWarning: This filename (/home/admincit/Desktop/Team_4/split_fif/mdd/sub-88000489_ses-1_task-restEC_eeg_1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_eo = mne.io.read_raw_fif(eo_file_path)
/tmp/ipykernel_2210216/113133525.py:11: RuntimeWarning: This filename (/home/admincit/Desktop/Team_4/split_fif/mdd/sub-88000489_ses-1_task-restEO_eeg_1.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  raw_ec = mne.io.read_raw_fif(ec_file_path)


Effective window size : 4.096 (s)
Features successfully saved to preprocessed.csv


# Function To process Folder

In [ ]:
import os

def process_folder(source_folder, destination_folder):
    """
    Processes all pairs of EO and EC files in the source folder and saves the combined features to CSV files in the destination folder.

    Args:
        source_folder (str): Path to the folder containing EEG files.
        destination_folder (str): Path to the folder where CSV files will be saved.
    """
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)

    files = os.listdir(source_folder)
    
    # Collecting EC and EO files
    ec_files = sorted([f for f in files if "restEC_eeg_1.fif" in f])
    eo_files = sorted([f for f in files if "restEO_eeg_1.fif" in f])

    for ec_file in ec_files:
        base_name = ec_file.replace("restEC_eeg_1.fif", "")
        eo_file = base_name + "restEO_eeg_1.fif"

        if eo_file in eo_files:
            ec_path = os.path.join(source_folder, ec_file)
            eo_path = os.path.join(source_folder, eo_file)
            output_path = os.path.join(destination_folder, base_name + "restcombined_eeg_1.csv")

            process_and_combine(ec_path, eo_path, output_path)
        else:
            print(f"Warning: No matching EO file found for {ec_file}")



In [ ]:
source_folder = "/home/admincit/Desktop/Team_4/split_fif/mdd"
destination_folder = "/home/admincit/Desktop/Team_4/split_fif/mdd_processed"
process_folder(source_folder, destination_folder)